# MQTT Publish 測試應用程式

這個 notebook 示範如何使用 paho-mqtt 套件來發佈 MQTT 訊息。


In [5]:
import paho.mqtt.client as mqtt
import time
import json


In [6]:
# MQTT Broker 設定
BROKER_HOST = "localhost"  # 連接到本地的 Raspberry Pi MQTT broker
# 或者使用 IP 地址（如果從其他設備連接）: "192.168.137.37"
BROKER_PORT = 1883  # 標準 MQTT 端口
TOPIC = "客廳/溫度"  # 發佈的主題名稱

# 可選：如果需要認證，取消註解並填入
# USERNAME = "your_username"
# PASSWORD = "your_password"


In [7]:
# 建立 MQTT 客戶端（使用最新的回調 API 版本 2）
client = mqtt.Client(callback_api_version=mqtt.CallbackAPIVersion.VERSION2)

# 可選：設定認證（如果有需要的話）
# client.username_pw_set(USERNAME, PASSWORD)

# 連接回調函數（API 版本 2 的簽名）
def on_connect(client, userdata, flags, reason_code, properties):
    if reason_code.is_failure:
        print(f"❌ 連接失敗，錯誤代碼: {reason_code}")
    else:
        print(f"✅ 成功連接到 MQTT Broker: {BROKER_HOST}")

# 發佈回調函數（API 版本 2 的簽名）
def on_publish(client, userdata, mid, reason_code, properties):
    print(f"📤 訊息已發佈，訊息 ID: {mid}")

# 設定回調函數
client.on_connect = on_connect
client.on_publish = on_publish


In [8]:
# 連接到 MQTT Broker
print(f"🔗 正在連接到 {BROKER_HOST}:{BROKER_PORT}...")
client.connect(BROKER_HOST, BROKER_PORT, 60)

# 開始網路循環（在背景處理訊息）
client.loop_start()

# 等待連接建立
time.sleep(1)


🔗 正在連接到 localhost:1883...
✅ 成功連接到 MQTT Broker: localhost


In [10]:
# 發佈簡單的文字訊息
message = "Hello MQTT from Raspberry Pi!"
result = client.publish(TOPIC, message, qos=0)
print(f"📨 發佈訊息到主題 '{TOPIC}': {message}")
print(f"   發佈狀態: {result.rc} (0=成功)")

time.sleep(0.5)  # 等待訊息發送


📨 發佈訊息到主題 '客廳/溫度': Hello MQTT from Raspberry Pi!
   發佈狀態: 0 (0=成功)
📤 訊息已發佈，訊息 ID: 2


In [11]:
# 發佈 JSON 格式的訊息
data = {
    "device": "Raspberry Pi",
    "temperature": 25.5,
    "humidity": 60,
    "timestamp": time.time()
}
json_message = json.dumps(data)
result = client.publish(TOPIC, json_message, qos=1)
print(f"📨 發佈 JSON 訊息到主題 '{TOPIC}':")
print(f"   {json.dumps(data, indent=2)}")
print(f"   發佈狀態: {result.rc} (0=成功)")

time.sleep(0.5)


📨 發佈 JSON 訊息到主題 '客廳/溫度':
   {
  "device": "Raspberry Pi",
  "temperature": 25.5,
  "humidity": 60,
  "timestamp": 1764474676.4273298
}
   發佈狀態: 0 (0=成功)
📤 訊息已發佈，訊息 ID: 3


In [12]:
# 連續發佈多筆測試訊息
print("🔄 開始連續發佈測試訊息...")
for i in range(5):
    test_message = f"測試訊息 #{i+1} - {time.strftime('%Y-%m-%d %H:%M:%S')}"
    result = client.publish(TOPIC, test_message, qos=0)
    print(f"   [{i+1}/5] {test_message}")
    time.sleep(1)  # 每秒發送一筆

print("✅ 所有測試訊息已發送完成！")


🔄 開始連續發佈測試訊息...
   [1/5] 測試訊息 #1 - 2025-11-30 11:51:24
📤 訊息已發佈，訊息 ID: 4
   [2/5] 測試訊息 #2 - 2025-11-30 11:51:25
📤 訊息已發佈，訊息 ID: 5
   [3/5] 測試訊息 #3 - 2025-11-30 11:51:26📤 訊息已發佈，訊息 ID: 6

   [4/5] 測試訊息 #4 - 2025-11-30 11:51:27📤 訊息已發佈，訊息 ID: 7

   [5/5] 測試訊息 #5 - 2025-11-30 11:51:28
📤 訊息已發佈，訊息 ID: 8
✅ 所有測試訊息已發送完成！


In [13]:
# 停止網路循環並斷開連接
client.loop_stop()
client.disconnect()
print("🔌 已斷開 MQTT 連接")


🔌 已斷開 MQTT 連接
